In [6]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import datetime
from sklearn.decomposition import PCA
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [2]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15, df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    # df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

def spread(df):
    # construct spread between home and away rankings
    ranks = [i for i in df.columns if 'RANK' in i]
    ranks_home = [i for i in ranks if '_x' in i]
    ranks_away = [i for i in ranks if '_y' in i]
    spread_columns_names = [col[:-1] for col in ranks_home]
    
    spread_columns = []
    for base_name in spread_columns_names:
        spread_column = df[f'{base_name}x'] - df[f'{base_name}y']
        spread_column.name = f'{base_name}spread'
        spread_columns.append(spread_column)
        
    spread_df = pd.concat(spread_columns, axis=1)
    return spread_df

In [20]:
!pwd

/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/models


In [54]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/raw_data/NBA_2018_2024.csv"

df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/2021_01_20_onward.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/odds_data/2021_01_20_onward.csv"
odds = pd.read_csv(folder_path, index_col=0)

In [55]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date'}, inplace=True)

#rename odds columns to match df
odds.rename(columns={'Timestamp': 'date_next'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp', 'winrate', 'winrate_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

df = df.dropna()

# scale the data for df
scaler = StandardScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

df = df.dropna()

# PCA


# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date', 'Teams']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = StandardScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# remove metadata for odds data
removed = list(odds.columns[odds.dtypes == 'object'])
valid_columns = odds.columns[~odds.columns.isin(removed)]

# scale the odds data
scaler = StandardScaler()
odds[valid_columns] = scaler.fit_transform(odds[valid_columns])

# construct current game metadata for df
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

# merge stats from nba dataframe
full = pd.merge(df, nba, on=['Teams', 'date'], how='left')
full = full.dropna()

# merge stats from opposing teams
complete = full.merge(full,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# concat the spreads
spread_df = spread(complete)
complete = pd.concat([complete, spread_df], axis=1)

# concat the odds
complete = pd.merge(complete, odds, on=['Teams_x', 'date_next', 'Teams_y'], how='left')
complete = complete.dropna()

pca = PCA(n_components=0.95)
disregard = list(complete.columns[complete.dtypes == 'object']) 
# disregard = disregard + ["target_x","target_y"]
# Temporary change for tuning
disregard = disregard + ["target_y" + "target_x"]
regard = complete.columns[~complete.columns.isin(disregard)]
new_data = pca.fit_transform(complete[regard])

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_56535/514699576.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_56535/514699576.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w4f

In [57]:
new_data.shape

(7284, 210)

In [69]:
# construct new dataset
new_data_df = pd.DataFrame(new_data)
complete_cleaning = pd.concat([new_data_df, complete['target_x']], axis=1)
complete_cleaning = complete_cleaning.dropna()

In [70]:
complete_cleaning

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,target_x
3480,38.932193,10.359131,3.423958,-10.867933,-10.191916,-4.527075,-5.092002,-8.047936,3.301930,8.520768,7.555385,-10.915409,-5.797639,6.786231,-2.310567,6.637961,0.888507,-6.441761,0.761253,-6.854159,-0.034790,6.724362,-5.482702,-3.885671,2.652593,-2.596124,-1.201480,2.018209,-3.577083,1.624797,6.938743,2.213901,-3.538007,0.489163,1.843563,3.610187,1.473977,1.626125,4.230126,-1.561270,0.718977,1.360121,3.752333,-3.850286,-0.658804,2.398301,-0.939845,0.849419,-0.314745,-0.849981,2.085697,-1.160466,0.292448,-1.085845,1.044355,-1.773182,1.712232,2.224505,0.882525,-1.251148,-0.954058,3.467242,-0.004625,-0.598897,0.053073,1.282731,-1.214162,0.694404,0.168475,1.516021,3.238554,-0.018992,-2.234965,-0.131055,1.407750,1.935900,-1.740643,0.895655,-0.440040,-2.138976,2.694747,1.220210,-0.051038,-0.625442,-1.415179,-0.161420,1.524471,1.220290,-1.577260,0.835839,0.565466,-2.181395,-1.615432,3.694167,0.959382,0.582909,-1.412699,-0.414199,1.713605,1.509660,-0.086495,-3.156318,-0.357320,-0.706386,2.061964,2.006242,2.150879,-0.681931,0.427717,1.171600,-3.072371,-2.977199,0.384891,-2.984971,1.270971,0.837872,-0.306985,0.409950,2.037288,-2.884417,0.342019,0.354760,-3.047873,0.288600,-3.413516,1.190697,1.033936,0.084777,0.782887,-0.460147,0.675773,3.605109,0.647844,1.236363,0.991778,-1.169503,0.835076,0.614633,-0.873964,0.700576,-2.206979,-0.108523,-1.504960,0.485185,0.255956,-0.477419,-0.854388,-0.727381,-0.241108,1.295797,-1.868560,-0.790290,-0.339685,-1.073472,-1.257389,-0.379514,1.054372,-1.353437,-0.288015,3.285049,0.132711,0.933371,2.857965,0.968239,1.379459,-0.861229,1.123858,-1.291637,-0.262946,-0.127624,1.962697,-0.922673,-0.483780,-0.710672,-0.822571,0.684370,-1.375738,0.574163,-0.155613,0.097246,0.086624,-1.164909,1.010678,-0.721900,1.459099,0.118124,-0.598240,-0.088634,-1.054098,0.331285,0.603434,-0.286857,-0.188738,0.188545,0.155689,0.952000,0.303764,0.290820,0.628234,-1.048564,0.900541,-1.649587,-0.664096,0.467994,0.321143,-0.498290,-0.293177,-0.630496,0.308141,-0.307566,1.0
3482,40.457300,16.374521,-6.923654,11.005788,-2.727756,-13.015506,-3.308281,1.097636,-2.729533,2.117121,4.624138,5.742022,-3.211445,-1.893318,-1.918782,8.846346,-5.749513,0.825835,-4.343874,0.324044,0.038228,4.008920,2.152770,3.428145,-0.594474,5.988318,-6.795020,0.544827,3.000067,-3.165540,-1.976002,0.102166,-0.646755,1.182642,-3.801903,2.239588,-1.798328,-2.053466,-2.600844,4.585037,0.346801,2.594669,-1.597531,-0.505415,-2.218588,0.164947,1.931386,5.542200,1.883463,-0.854850,-0.165563,0.904898,4.071357,0.549065,0.717642,0.564795,0.739862,-2.136789,0.894084,-0.448777,-2.262250,-0.023272,-2.020136,2.561427,-1.314704,0.309107,0.480301,2.281675,1.796859,3.095455,0.459063,0.070807,-0.882637,-3.331787,1.065631,-0.930052,1.483801,-2.089211,2.350446,2.173723,-1.340735,0.960964,-1.328044,-0.163234,2.116012,-1.439411,-0.040280,-0.633866,0.157461,-1.748924,1.895237,-1.451069,0.357742,0.643184,0.080198,0.455668,-0.819971,0.265922,1.426194,-0.278167,-0.508526,0.187272,-0.598379,2.608008,0.186063,1.591676,1.333410,-0.132662,0.448536,1.816544,-0.990111,1.230279,2.157779,-0.336482,0.521620,-0.400394,0.090261,0.979838,1.402480,0.187318,-0.361730,0.095007,-1.236180,-0.115535,-1.337541,1.196263,-0.349386,0.938315,-0.185216,-1.436374,-2.094814,0.460247,-

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier
from scipy.stats import uniform, randint

# Assuming complete_cleaning is your DataFrame ready for use
features = [col for col in complete_cleaning.columns if col != 'target_x']
X = complete_cleaning[features]
y = complete_cleaning['target_x']

# Initialize the RidgeClassifier
rr = RidgeClassifier()

# Create a pipeline with Sequential Feature Selector and RidgeClassifier
pipeline = Pipeline([
    ('feature_selector', SequentialFeatureSelector(estimator=rr, direction='forward')),
    ('classifier', rr)
])

# Define the parameter distributions
param_distributions = {
    'feature_selector__n_features_to_select': randint(1, len(features) + 1),  # Correct inclusion of n_features_to_select
    'classifier__alpha': uniform(0.1, 10),  # Regularization strength
    'classifier__solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']  # Solver
}

# Initialize TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=100,  # Number of parameter settings sampled
    cv=tscv,  # Cross-validation strategy
    verbose=1,
    n_jobs=-1
)

# Perform the randomized search
random_search.fit(X, y)

# Output the best combination of parameters
best_hyperparams = random_search.best_params_
print("Best hyperparameters:", best_hyperparams)

# Evaluate the best model further if necessary, for example on a test set


Fitting 5 folds for each of 100 candidates, totalling 500 fits
